In [ ]:
!pip install albumentations opencv-python-headless


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 369.4/369.4 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.8/307.8 kB 21.3 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import uuid
import hashlib
from glob import glob
import albumentations as A
from google.colab import drive


In [ ]:
drive.mount('/content/drive')
BASE_DIR = '/content/drive/MyDrive/stain'
NORMAL_DIR = os.path.join(BASE_DIR, 'normal')
# DEFECT_DIR = os.path.join(BASE_DIR, 'defect')
# NORMAL_DIR = os.path.join(BASE_DIR, 'contaminated')
# DEFECT_DIR = os.path.join(BASE_DIR, 'line')
# NORMAL1_DIR = os.path.join(BASE_DIR, 'string')
# DEFECT1_DIR = os.path.join(BASE_DIR, 'flecked')

Mounted at /content/drive


In [ ]:
augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3),
    A.GaussianBlur(blur_limit=3, p=0.3),
    A.ElasticTransform(p=0.2),
    A.ShiftScaleRotate(
        shift_limit=0.05, scale_limit=0.08, rotate_limit=15,
        border_mode=cv2.BORDER_REFLECT_101, p=0.5
    )
])


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [ ]:
def image_hash(img):
    """Compute a hash for an image array to detect duplicates."""
    return hashlib.md5(img.tobytes()).hexdigest()


In [ ]:
# def augment_folder(src_dir, tgt_dir, target_total):
#     os.makedirs(tgt_dir, exist_ok=True)
#     # Load existing images and their hashes
#     existing_paths = glob(os.path.join(tgt_dir, '*'))
#     hashes = set()
#     for p in existing_paths:
#         img = cv2.imread(p)
#         if img is not None:
#             hashes.add(image_hash(img))

#     current_count = len(existing_paths)
#     needed = target_total - current_count
#     print(f"{os.path.basename(src_dir)}: {current_count} existing, need {needed} more")

#     src_paths = glob(os.path.join(src_dir, '*'))
#     i = 0
#     while needed > 0:
#         # cycle through source images
#         src_img_path = src_paths[i % len(src_paths)]
#         img = cv2.imread(src_img_path)
#         if img is None:
#             i += 1
#             continue

#         transformed = augmentation(image=img)['image']
#         h = image_hash(transformed)
#         if h in hashes:
#             # skip duplicates
#             i += 1
#             continue

#         # save uniquely
#         out_name = f"{uuid.uuid4().hex}.png"
#         cv2.imwrite(os.path.join(tgt_dir, out_name), transformed)
#         hashes.add(h)
#         needed -= 1
#         i += 1

#     print(f"Finished augmenting {os.path.basename(src_dir)}; now {target_total} images.")


import os
import cv2
import uuid
from glob import glob

def augment_folder(src_dir, tgt_dir, target_total):
    os.makedirs(tgt_dir, exist_ok=True)

    # Load existing images and their hashes
    existing_paths = glob(os.path.join(tgt_dir, '*'))
    hashes = set()
    for p in existing_paths:
        img = cv2.imread(p)
        if img is not None:
            hashes.add(image_hash(img))

    current_count = len(existing_paths)
    needed = target_total - current_count
    print(f"{os.path.basename(src_dir)}: {current_count} existing, need {needed} more")

    src_paths = glob(os.path.join(src_dir, '*'))
    i = 0
    duplicate_count = 0  # Track how many duplicates were skipped

    while needed > 0:
        # cycle through source images
        src_img_path = src_paths[i % len(src_paths)]
        img = cv2.imread(src_img_path)
        if img is None:
            i += 1
            continue

        transformed = augmentation(image=img)['image']
        h = image_hash(transformed)
        if h in hashes:
            duplicate_count += 1  # Count duplicates
            i += 1
            continue

        # save uniquely
        out_name = f"{uuid.uuid4().hex}.png"
        cv2.imwrite(os.path.join(tgt_dir, out_name), transformed)
        hashes.add(h)
        needed -= 1
        i += 1

    print(f"Finished augmenting {os.path.basename(src_dir)}; now {target_total} images.")
    print(f"Skipped {duplicate_count} duplicate augmented images.")



In [ ]:
# # Normal: from 320 → 500 (80 more)
augment_folder(NORMAL_DIR, NORMAL_DIR, target_total=398)

# # Defect: from 32 → 500 (320 more)
# augment_folder(DEFECT_DIR, DEFECT_DIR, target_total=500)

# # # Normal: from 320 → 500 (80 more)
# augment_folder(NORMAL_DIR, NORMAL_DIR, target_total=125)

# # Defect: from 32 → 500 (320 more)
# augment_folder(DEFECT_DIR, DEFECT_DIR, target_total=125)
# # # Normal: from 320 → 500 (80 more)
# augment_folder(NORMAL_DIR, NORMAL1_DIR, target_total=125)

# # Defect: from 32 → 500 (320 more)
# augment_folder(DEFECT_DIR, DEFECT1_DIR, target_total=125)


normal: 68 existing, need 330 more
Finished augmenting normal; now 398 images.
Skipped 24 duplicate augmented images.
